In [9]:
#MultiClass classifier. Predicts one attribute as one of many labels rather than just 2 possible labels.
#This assumes there is only one right answer. We could do multilabel classification and present multiple city solutions to the user?
#https://scikit-learn.org/stable/modules/multiclass.html    a look at different kinds of models
# Import necessary librarie s
#     ut.run_script(re.DB_WEATHER_DESC_SCRIPT)     copy this kind of thing to add mountains/water data

# method of going through multiple models
# https://medium.com/analytics-vidhya/testing-multiple-machine-learning-models-at-once-without-getting-a-headache-5aefb0e7df03    simple good
# https://github.com/j-planet/machine-learning-big-loop   more advanced, but better. (get parameters.py from this, and the cell for running multiple things


# Import necessary modules from repository
from model.data.pipeline import aggregate

In [10]:
df = aggregate.get_agg_data_frame()

df

,city,year,month,temperature,humidity,pressure,wind_speed,rain,snow,cloud,fog,extreme_weather
0,Albuquerque,2012,10,282.594630,40.870130,1017.681818,1.980519,0.006431,0.000000,0.456592,0.000000,0.000000
1,Albuquerque,2012,11,276.787692,44.012821,994.980769,1.801282,0.019802,0.001650,0.526403,0.003300,0.003300
2,Albuquerque,2012,12,271.423110,55.940476,1010.517857,1.821429,0.010401,0.001486,0.754829,0.001486,0.004458
3,Albuquerque,2013,01,268.405723,57.488372,999.837209,2.453488,0.036023,0.002882,0.576369,0.011527,0.001441
4,Albuquerque,2013,02,271.623359,54.910256,953.775641,2.275641,0.011024,0.007874,0.587402,0.001575,0.009449
...,...,...,...,...,...,...,...,...,...,...,...,...
1666,Seattle,2017,07,286.727283,84.059140,1020.053763,1.607527,0.020161,0.000000,0.442204,0.081989,0.000000
1667,Seattle,2017,08,287.951183,84.413978,1017.043011,1.553763,0.025538,0.000000,0.302419,0.290323,0.106183
1668,Seattle,2017,09,285.830661,87.100000,1015.833333,1.738889,0.181944,0.000000,0.465278,0.226389,0.043056
1669,Seattle,2017,10,280.271667,92.629032,1019.784946,2.048387,0.231183,0.000000,0.475806,0.314516,0.002688


In [11]:
labels = df["city"]
features = df.drop(['city', 'year'], axis=1)

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
featPrepped = scaler.fit_transform(features)

In [140]:
from sklearn.model_selection import train_test_split

feat_train, feat_test, lab_train, lab_test = train_test_split(featPrepped, labels, random_state=42)
# stratify by city for separating training and testing. stratified sampling

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

clf = RandomForestClassifier()
clfFit = clf.fit(feat_train, lab_train)

print(clf.score(feat_test, lab_test))
# low scare shows we do not have enough destinctions between cities.
# Score got worse when we expanded the data by years. went from like 47 % to 35 %


results = roc_auc_score(lab_train, clfFit.predict_proba(feat_train), multi_class='ovr')

0.3827751196172249


array([[-1.33251985, -1.92582103,  0.03944503, ...,  1.29910543,
        -0.42374686,  3.36147883],
       [ 0.97428219, -0.19624353,  0.40257645, ...,  0.12868071,
        -0.61223384, -0.3110261 ],
       [ 0.10923142,  1.08339226,  0.02555013, ..., -0.3265737 ,
        -0.20574805, -0.3307394 ],
       ...,
       [-0.17911883,  0.94299197, -0.57337001, ...,  0.60201125,
        -0.46504851, -0.27197296],
       [-0.46746908,  0.31552777, -0.12088069, ...,  1.49761431,
         0.17296927, -0.10675124],
       [-1.33251985, -0.97478081,  1.21833442, ...,  0.98996644,
         1.36742067, -0.52906509]])

In [170]:
import pandas as pd
import numpy

pred_proba = clfFit.predict_proba(feat_test)
print("Accuracy:", clfFit.score(feat_test, lab_test))

df = pd.DataFrame(pred_proba)
df


Accuracy: 0.3708133971291866


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,0.01,0.00,0.00,0.02,0.18,0.03,0.02,0.06,0.00,0.13,...,0.01,0.01,0.00,0.16,0.00,0.05,0.00,0.00,0.01,0.00
1,0.01,0.02,0.07,0.02,0.05,0.01,0.06,0.08,0.04,0.08,...,0.06,0.05,0.00,0.19,0.00,0.03,0.05,0.00,0.00,0.00
2,0.00,0.08,0.06,0.04,0.01,0.08,0.01,0.11,0.00,0.09,...,0.01,0.11,0.03,0.00,0.00,0.04,0.09,0.02,0.00,0.00
3,0.00,0.03,0.08,0.03,0.03,0.07,0.00,0.10,0.17,0.06,...,0.00,0.01,0.00,0.01,0.04,0.08,0.05,0.00,0.04,0.13
4,0.00,0.03,0.00,0.04,0.01,0.00,0.00,0.00,0.03,0.01,...,0.01,0.01,0.02,0.02,0.09,0.00,0.01,0.22,0.03,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.04,0.05,0.00,0.01,0.01,0.00,0.01,0.00,0.00,0.04,...,0.00,0.02,0.02,0.03,0.14,0.03,0.03,0.14,0.12,0.22
414,0.01,0.05,0.04,0.08,0.01,0.02,0.15,0.16,0.00,0.01,...,0.11,0.08,0.00,0.08,0.00,0.05,0.00,0.01,0.01,0.00
415,0.01,0.01,0.03,0.03,0.13,0.12,0.02,0.13,0.00,0.05,...,0.08,0.09,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00
416,0.00,0.00,0.01,0.00,0.21,0.05,0.00,0.15,0.00,0.09,...,0.11,0.00,0.00,0.06,0.04,0.01,0.01,0.00,0.00,0.01


In [188]:
import numpy as np
TOP = 5
TMP = 62
CITIES = labels.unique()

predictions_list = []
for i in range(len(df)):
    row = df.iloc[i].sort_values(ascending=False)[:TOP]
    rowDf = row.to_frame().reset_index()
    #print([[CITIES[int(x[0])], x[1]] for x in rowDf.to_numpy()])
    vals = np.insert(np.array([[CITIES[int(x[0])], x[1]] for x in rowDf.to_numpy()]), 0, lab_test.iloc[i])
    predictions_list.append(vals)

cols = ['true']
for i in range(TOP):
    cols.append('city{x}'.format(x=i + 1))
    cols.append('prob{x}'.format(x=i + 1))

preds = pd.DataFrame(predictions_list,
                     columns=cols)
preds


,true,city1,prob1,city2,prob2,city3,prob3,city4,prob4,city5,prob5
0,Detroit,Minneapolis,0.27,Chicago,0.18,Pittsburgh,0.16,Indianapolis,0.13,Detroit,0.06
1,Indianapolis,Pittsburgh,0.19,Minneapolis,0.14,Detroit,0.08,Indianapolis,0.08,Boston,0.07
2,Indianapolis,Philadelphia,0.11,Detroit,0.11,San Antonio,0.09,Indianapolis,0.09,Dallas,0.08
3,Houston,Houston,0.17,Seattle,0.13,Detroit,0.1,Boston,0.08,Saint Louis,0.08
4,Seattle,Seattle,0.27,San Diego,0.22,Nashville,0.1,Portland,0.09,Charlotte,0.04
...,...,...,...,...,...,...,...,...,...,...,...
413,San Diego,Seattle,0.22,San Diego,0.14,Portland,0.14,San Francisco,0.12,Atlanta,0.05
414,Philadelphia,Detroit,0.16,Denver,0.15,New York,0.11,Charlotte,0.08,Pittsburgh,0.08
415,Chicago,Chicago,0.13,Detroit,0.13,Dallas,0.12,Philadelphia,0.09,Kansas City,0.08
416,Indianapolis,Chicago,0.21,Detroit,0.15,New York,0.11,Indianapolis,0.09,Miami,0.08


In [193]:
count = 0
for i in range(len(preds)):
    if preds.iloc[i]['true'] == preds.iloc[i]['city1']:
        count += 1
        #print('{i}: {t} and {p}'.format(i=i, t=preds['true'][i], p=preds['city1'][i]))

print('Accuracy: {r}/{t}, {p}%'.format(r=count, t=len(preds), p=round(100*count/len(preds))))

Accuracy: 154/418, 37%


In [205]:
CITY = 'Denver'

for i in range(len(preds)):
    row = preds.iloc[i]
    if row['true'] == CITY and row['city1'] != CITY:
        print(row['city1'])

Pittsburgh
Philadelphia
Albuquerque
Albuquerque
Boston
Saint Louis
Saint Louis
Albuquerque
Boston
Albuquerque
Philadelphia
New York


In [200]:
CITIES

array(['Albuquerque', 'Atlanta', 'Boston', 'Charlotte', 'Chicago',
       'Dallas', 'Denver', 'Detroit', 'Houston', 'Indianapolis',
       'Jacksonville', 'Kansas City', 'Las Vegas', 'Los Angeles', 'Miami',
       'Minneapolis', 'Nashville', 'New York', 'Philadelphia', 'Phoenix',
       'Pittsburgh', 'Portland', 'Saint Louis', 'San Antonio',
       'San Diego', 'San Francisco', 'Seattle'], dtype=object)

In [15]:








from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(feat_train, lab_train)

clf.score(feat_test, lab_test)

0.2822966507177033

```
SELECT DISTINCT weather_description
FROM USWeather;

SELECT <columns>
FROM <table>
#WHERE <condition>
#ORDER BY <column> <asc/desc>;
```

In [17]:
from sklearn.svm import SVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid, RadiusNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

log_reg_params = [{"C": 0.01}, {"C": 0.1}, {"C": 1}, {"C": 10}]
dec_tree_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
rand_for_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
kneighbors_params = [{"n_neighbors": 3}, {"n_neighbors": 5}]
naive_bayes_params = [{}]
svc_params = [{"C": 0.01}, {"C": 0.1}, {"C": 1}, {"C": 10}]

modelclasses = [
    ["GradientBoostingClf", GradientBoostingClassifier()],
    ["ExtraTreeClf", ExtraTreesClassifier()],
    ["decision tree", DecisionTreeClassifier()],
    ["random forest", RandomForestClassifier()],
    ["k neighbors", KNeighborsClassifier()],
    ["NearestCentroid", NearestCentroid()],
    #["RadiusNeighbor", RadiusNeighborsClassifier()],
    ["GaussianProcess", GaussianProcessClassifier()],
    ["naive bayes Gaussian NB", GaussianNB()],
    ["naive bayes Bernoulli NB", BernoulliNB()],
    ["support vector machines", SVC()],
    ["Nu support vector machines", NuSVC()],
    ["NN MPL", MLPClassifier()]
]

insights = []
for modelname, model in modelclasses:
    model.fit(feat_train, lab_train)
    score = model.score(feat_test, lab_test)
    insights.append((modelname, model, score))

insights.sort(key=lambda x: x[-1], reverse=True)
for modelname, model, score in insights:
    print(modelname, score)





NN MPL 0.39952153110047844
Nu support vector machines 0.38516746411483255
random forest 0.361244019138756
ExtraTreeClf 0.3588516746411483
GradientBoostingClf 0.3492822966507177
support vector machines 0.3492822966507177
GaussianProcess 0.3397129186602871
k neighbors 0.29904306220095694
NearestCentroid 0.2727272727272727
decision tree 0.22248803827751196
naive bayes Bernoulli NB 0.21770334928229665
naive bayes Gaussian NB 0.19138755980861244


c:\users\trenton\documents\programming\ml-team3\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
